In [112]:
import mysql.connector
from tabulate import tabulate
import pandas as pd
import math
import sys

## Connect to Remote Database
## Insert database information

HOST = "147.46.15.238"
PORT = ""
USER = "DS2024_0024"
PASSWD = "DS2024_0024"
DB = "DS_proj_12"

connection = mysql.connector.connect(
    host=HOST,
    port=7000,
    user=USER,
    passwd=PASSWD,
    db=DB,
    autocommit=True  # to create table permanently
)

cur = connection.cursor(dictionary=True)

## 수정할 필요 없는 함수입니다.
# DO NOT CHANGE INITIAL TABLES IN prj.sql
def get_dump(mysql_con, filename):
    '''
    connect to mysql server using mysql_connector
    load .sql file (filename) to get queries that create tables in an existing database (fma)
    '''
    query = ""
    try:
        with mysql_con.cursor() as cursor:
            for line in open(filename, 'r'):
                if line.strip():
                    line = line.strip()
                    if line[-1] == ";":
                        query += line
                        cursor.execute(query)
                        query = ""
                    else:
                        query += line

    except Warning as warn:
        print(warn)
        sys.exit()


def get_output(query):
    cur.execute(query)
    out = cur.fetchall()
    df = pd.DataFrame(out)
    return df

get_dump(connection, 'prj.sql')

In [116]:
query ='SELECT F.item, F.prediction\
    	FROM (SELECT A.item, ROUND(avg(A.scaled_rating),4) AS prediction\
				FROM (SELECT user, item, (rating-min)/(max-min) AS scaled_rating\
						FROM ratings AS R NATURAL JOIN (SELECT user,\
														CASE count(rating)\
																WHEN 1 then 0\
														ELSE min(rating)\
														END AS min,\
														max(rating) AS max\
														FROM ratings\
														GROUP BY user) AS S\
						) AS A\
				GROUP BY A.item\
				ORDER BY prediction DESC) AS F\
        WHERE F.item >= 150 AND F.item < 350'

get_output(query)

,item,prediction
0,316,0.8666
1,203,0.8634
2,307,0.8347
3,290,0.8321
4,310,0.8311
...,...,...
195,151,0.4378
196,271,0.3591
197,264,0.3576
198,285,0.3413


In [54]:
query ='SELECT A.item, ROUND(avg(A.scaled_rating),4) AS avg_rating\
    	FROM (SELECT user, item, (rating-min)/(max-min) AS scaled_rating\
			  FROM ratings AS R NATURAL JOIN (SELECT user,\
											CASE count(rating)\
													WHEN 1 then 0\
											ELSE min(rating)\
											END AS min,\
											max(rating) AS max\
											FROM ratings\
											GROUP BY user) AS S\
			 ) AS A\
        GROUP BY A.item\
		ORDER BY avg_rating DESC'
df = get_output(query)


In [55]:
df

,item,avg_rating
0,316,0.8666
1,203,0.8634
2,439,0.8528
3,69,0.8458
4,4,0.8427
...,...,...
448,451,0.3215
449,450,0.3206
450,58,0.3096
451,63,0.2929


In [56]:
query = 'SHOW tables'
get_output(query)

,Tables_in_DS_proj_12
0,item_similarity
1,ratings
2,user_similarity


## algo 3

### 아이템 유사도

In [117]:
rec_num = 8

In [173]:
query = 'SELECT * FROM item_similarity'
get_output(query).head(20)

,item_1,item_2,sim
0,0,0,0.0
1,0,1,0.6
2,0,2,0.7
3,0,3,0.8
4,0,4,0.7
5,0,5,0.6
6,0,6,0.7
7,0,7,0.8
8,0,8,0.8
9,0,9,0.8


In [172]:
query = 'SELECT item_1, sum(sim) AS sim_sum \
		FROM (SELECT item_1, item_2, sim, ROW_NUMBER() OVER(PARTITION BY item_1 ORDER BY sim DESC, item_2 ASC) AS ranking\
			FROM item_similarity) AS A\
        WHERE A.ranking <= 5\
        GROUP BY item_1'
get_output(query).head(20)

,item_1,sim_sum
0,0,4.0
1,1,3.9
2,2,4.0
3,3,4.2
4,4,4.1
5,5,3.6
6,6,3.6
7,7,4.2
8,8,4.0
9,9,4.1


In [170]:
query2 = 'SELECT I.item_1, I.item_2, ROUND(I.sim/S.sim_sum , 4) AS nor_sim\
		FROM (SELECT item_1, item_2, sim, ROW_NUMBER() OVER(PARTITION BY item_1 ORDER BY sim DESC, item_2 ASC) AS ranking\
			FROM item_similarity) AS I NATURAL JOIN ('+ query +') AS S\
        WHERE I.ranking <= 5'
get_output(query2)

,item_1,item_2,nor_sim
0,0,3,0.2000
1,0,7,0.2000
2,0,8,0.2000
3,0,9,0.2000
4,0,10,0.2000
...,...,...,...
2260,452,1,0.2000
2261,452,5,0.2000
2262,452,6,0.2000
2263,452,7,0.2000


In [120]:
query3 = 'SELECT L.item_1, L.item_2,  CASE \
					WHEN R.nor_sim is null THEN 0\
					ELSE R.nor_sim\
					END AS chg_sim\
    	FROM item_similarity AS L LEFT OUTER JOIN (' + query2 + ') AS R USING (item_1, item_2)\
            '
item_sim = get_output(query3)
item_sim

,item_1,item_2,chg_sim
0,0,0,0
1,0,1,0
2,0,2,0
3,0,3,0.2000
4,0,4,0
...,...,...,...
205204,452,448,0
205205,452,449,0
205206,452,450,0
205207,452,451,0


### 아이템-사용자

In [125]:
query = 'SELECT R.item, R.user,\
			CASE \
				WHEN R.rating is null THEN T.rate_avg\
			ELSE R.rating\
			END AS rate\
    	FROM ratings AS R NATURAL JOIN (SELECT item, avg(rating) AS rate_avg\
										FROM ratings\
										GROUP BY item) AS T'

item_user = get_output(query)
item_user

,item,user,rate
0,0,0,4.00
1,1,0,5.00
2,2,0,5.00
3,3,0,4.00
4,4,0,4.00
...,...,...,...
132271,448,291,3.215686
132272,449,291,3.656250
132273,450,291,2.571429
132274,451,291,2.467742


### 행렬곱
item_sim , item_user

In [13]:
item_user

,item,user,rate
0,0,0,4.00
1,1,0,5.00
2,2,0,5.00
3,3,0,4.00
4,4,0,4.00
...,...,...,...
132271,448,291,3.215686
132272,449,291,3.656250
132273,450,291,2.571429
132274,451,291,2.467742


In [123]:
ii_pivot = pd.pivot_table(item_sim, index= 'item_1', columns= 'item_2', values= 'chg_sim')
ii_pivot

item_2,0,1,2,3,4,5,6,7,8,9,...,443,444,445,446,447,448,449,450,451,452
item_1,,,,,,,,,,,,,,,,,,,,,
0,0.0,0.0,0.0,0.2,0.0,0.0,0.0,0.2,0.2,0.2,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.1795,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.1905,0.0,0.0,0.0,0.1905,0.0,0.0,0.1905,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.1951,0.0,0.0,0.0,0.1951,0.1951,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
448,0.0,0.1842,0.1842,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
449,0.0,0.0,0.0,0.1944,0.1944,0.0,0.0,0.1944,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
450,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.2,0.0,0.0,0.0,0.0,0.0,0.0


In [126]:
iu_pivot = pd.pivot_table(item_user, index='item', columns='user', values='rate')
iu_pivot

user,0,1,2,3,4,5,6,7,8,9,...,282,283,284,285,286,287,288,289,290,291
item,,,,,,,,,,,,,,,,,,,,,
0,4.0,4.0,3.36413,3.0,3.36413,3.0,3.36413,3.0,3.36413,3.0,...,3.36413,3.36413,5.0,4.0,3.36413,3.36413,3.0,3.36413,3.36413,3.36413
1,5.0,3.990566,3.990566,3.990566,3.990566,3.0,3.990566,3.990566,3.990566,3.990566,...,3.990566,3.990566,3.990566,4.5,3.990566,4.0,3.990566,3.990566,3.990566,3.990566
2,5.0,3.526316,4.0,3.526316,3.526316,2.5,3.526316,3.0,3.526316,3.526316,...,3.526316,3.526316,4.0,3.526316,3.526316,3.526316,3.0,3.526316,2.0,3.526316
3,4.0,4.036913,4.036913,4.036913,5.0,5.0,5.0,5.0,4.036913,4.0,...,5.0,3.0,4.036913,4.5,4.036913,4.0,3.0,4.5,4.036913,4.036913
4,4.0,4.37931,4.37931,4.37931,5.0,5.0,5.0,4.0,4.37931,4.37931,...,5.0,4.37931,4.0,5.0,5.0,4.0,4.37931,4.5,4.37931,4.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
448,3.215686,3.215686,3.215686,3.215686,3.215686,3.215686,3.215686,3.215686,3.215686,3.215686,...,3.215686,3.215686,3.215686,4.0,3.215686,3.215686,3.215686,3.215686,3.215686,3.215686
449,3.65625,3.65625,3.65625,3.65625,3.65625,3.65625,3.65625,3.65625,3.65625,3.65625,...,3.65625,3.65625,3.65625,3.65625,3.65625,3.65625,3.65625,4.0,3.65625,3.65625
450,2.571429,2.571429,2.571429,2.571429,2.571429,2.571429,2.571429,2.571429,2.571429,2.571429,...,2.571429,2.571429,2.571429,2.571429,2.571429,2.571429,3.0,2.571429,2.571429,2.571429


In [127]:
dot_df = ii_pivot.dot(iu_pivot)
dot_df

user,0,1,2,3,4,5,6,7,8,9,...,282,283,284,285,286,287,288,289,290,291
item_1,,,,,,,,,,,,,,,,,,,,,
0,4.0,3.924074,3.663187,3.862991,3.855608,3.0,3.841411,3.545283,3.352666,3.641411,...,3.696128,3.441411,3.993185,4.4,3.648793,3.745283,3.455805,3.755608,3.648793,3.648793
1,4.002864,4.196988,3.707845,3.581688,3.622819,2.49975,3.622819,3.772243,3.616577,3.719127,...,3.972083,3.622819,3.823364,4.004155,3.622819,3.328137,3.487215,3.622819,2.692415,3.485381
2,3.2,4.675325,3.871138,3.337117,3.74908,3.2,3.64908,4.4,2.609345,3.54908,...,3.952178,3.709345,3.6,4.334021,3.912443,4.0,3.6,4.04908,3.6,3.81506
3,3.7861,4.309152,4.039929,4.174586,4.176917,4.2385,4.377694,3.6194,2.070425,3.654335,...,4.484844,4.188019,4.605977,4.6076,4.09196,3.688767,3.690052,4.295967,3.938002,3.99671
4,3.6094,4.200603,4.062961,4.25787,4.182851,4.024,4.093659,4.024,2.655444,3.754401,...,4.388501,3.998301,4.212452,4.8044,3.981103,3.782292,3.573342,4.302609,3.981103,3.981103
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
448,4.184124,4.54211,3.813986,3.726734,3.654035,3.43385,3.726734,3.629786,3.726734,3.726734,...,3.726734,3.726734,3.813986,4.214947,3.726734,3.728471,3.629786,3.726734,3.418962,3.759285
449,3.407949,4.290698,4.060447,4.276698,4.382134,3.9714,3.8604,3.9992,3.685447,4.283522,...,4.368333,3.858871,3.984525,4.6935,4.181109,3.798934,3.858871,4.1936,4.060447,4.09196
450,2.924923,3.21566,3.005801,3.005801,3.005801,1.634783,3.005801,2.642744,3.005801,3.032885,...,3.005801,3.005801,3.147699,3.634783,3.005801,3.005801,3.172917,3.005801,3.403057,2.971019


In [160]:
df_mid = dot_df.stack().reset_index()

In [161]:
df_mid[0] = df_mid[0].astype(float).round(4)

In [162]:
df_nd = df_mid.rename(columns={'item_1':'item',0:'prediction'}).sort_values(['user','prediction','item'], ascending=[True,False,True])
df_nd = df_nd[df_nd.item.isin(or_list)]

In [164]:
df_nd.groupby('user').head(rec_num)[['user','item','prediction']].reset_index(drop=True)

,user,item,prediction
0,0,56,4.6000
1,0,40,4.4000
2,0,47,4.4000
3,0,63,4.4000
4,0,383,4.3713
...,...,...,...
2331,291,122,4.6729
2332,291,89,4.6028
2333,291,106,4.5765
2334,291,118,4.5679


In [145]:
df_fin = df_mid.rename(columns={'item_1':'item',0:'prediction'}).sort_values(['user','prediction','item'], ascending=[True,False,True]).groupby('user').head(rec_num)[['user','item','prediction']].reset_index(drop=True)
df_fin

,user,item,prediction
0,0,56,4.6000
1,0,40,4.4000
2,0,47,4.4000
3,0,63,4.4000
4,0,383,4.3713
...,...,...,...
2331,291,97,4.7000
2332,291,122,4.6729
2333,291,89,4.6028
2334,291,106,4.5765


In [148]:
query = 'SELECT * FROM ratings'

origin = get_output(query)
origin

,user,item,rating
0,0,0,4.00
1,0,1,5.00
2,0,2,5.00
3,0,3,4.00
4,0,4,4.00
...,...,...,...
132271,291,448,None
132272,291,449,None
132273,291,450,None
132274,291,451,None


In [155]:
or_list = origin[(origin.user == 2) & (origin.rating.isnull())].item.to_list()

## ALGO 4

### 유저 유사도

In [175]:
rec_num = 7
user = 3

In [176]:
query = 'SELECT user_1, sum(sim) AS sim_sum \
		FROM (SELECT user_1, user_2, sim, ROW_NUMBER() OVER(PARTITION BY user_1 ORDER BY sim DESC, user_2 ASC) AS ranking\
			FROM user_similarity) AS A\
        WHERE A.ranking <= 5\
        GROUP BY user_1'
get_output(query)

,user_1,sim_sum
0,0,4.0
1,1,4.0
2,2,3.5
3,3,3.8
4,4,4.0
...,...,...
287,287,4.0
288,288,4.4
289,289,4.1
290,290,4.0


In [177]:
query2 = 'SELECT I.user_1, I.user_2, ROUND(I.sim/S.sim_sum , 4) AS nor_sim\
		FROM (SELECT user_1, user_2, sim, ROW_NUMBER() OVER(PARTITION BY user_1 ORDER BY sim DESC, user_2 ASC) AS ranking\
			FROM user_similarity) AS I NATURAL JOIN ('+ query +') AS S\
        WHERE I.ranking <= 5'
get_output(query2)

,user_1,user_2,nor_sim
0,0,35,0.2000
1,0,36,0.2000
2,0,40,0.2000
3,0,49,0.2000
4,0,54,0.2000
...,...,...,...
1455,291,59,0.2000
1456,291,189,0.2000
1457,291,201,0.2000
1458,291,202,0.2000


In [178]:
query3 = 'SELECT L.user_1, L.user_2,  CASE \
					WHEN R.nor_sim is null THEN 0\
					ELSE R.nor_sim\
					END AS chg_sim\
    	FROM user_similarity AS L LEFT OUTER JOIN (' + query2 + ') AS R USING (user_1, user_2)\
            '
user_sim = get_output(query3)
user_sim

,user_1,user_2,chg_sim
0,0,0,0
1,0,1,0
2,0,2,0
3,0,3,0
4,0,4,0
...,...,...,...
85259,291,287,0
85260,291,288,0
85261,291,289,0
85262,291,290,0


### 사용자-아이템

In [183]:
query = 'SELECT R.user, R.item,\
			CASE \
				WHEN R.rating is null THEN T.rate_avg\
			ELSE R.rating\
			END AS rate\
    	FROM ratings AS R NATURAL JOIN (SELECT user, avg(rating) AS rate_avg\
										FROM ratings\
										GROUP BY user) AS T'

user_item = get_output(query)
user_item

,user,item,rate
0,0,0,4.00
1,0,1,5.00
2,0,2,5.00
3,0,3,4.00
4,0,4,4.00
...,...,...,...
132271,291,448,4.010638
132272,291,449,4.010638
132273,291,450,4.010638
132274,291,451,4.010638


### 피봇, 행렬곱

In [184]:
uu_pivot = pd.pivot_table(user_sim, index= 'user_1', columns= 'user_2', values= 'chg_sim')
uu_pivot

user_2,0,1,2,3,4,5,6,7,8,9,...,282,283,284,285,286,287,288,289,290,291
user_1,,,,,,,,,,,,,,,,,,,,,
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.2,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.2,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.1842,0.0,0.0,0.0,0.1842,0.0,0.2105,0.0,0.2105,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
287,0.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
288,0.1818,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
289,0.0,0.0,0.0,0.0,0.0,0.1951,0.1951,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [185]:
ui_pivot = pd.pivot_table(user_item, index='user', columns='item', values='rate')
ui_pivot

item,0,1,2,3,4,5,6,7,8,9,...,443,444,445,446,447,448,449,450,451,452
user,,,,,,,,,,,,,,,,,,,,,
0,4.0,5.0,5.0,4.0,4.0,3.0,3.0,4.0,5.0,4.0,...,3.535714,3.535714,3.535714,3.535714,3.535714,3.535714,3.535714,3.535714,3.535714,3.535714
1,4.0,4.556604,4.556604,4.556604,4.556604,4.556604,4.556604,4.556604,4.556604,4.0,...,4.556604,4.556604,4.556604,4.556604,4.556604,4.556604,4.556604,4.556604,4.556604,4.556604
2,3.881579,3.881579,4.0,3.881579,3.881579,3.881579,3.881579,3.881579,4.0,3.881579,...,3.881579,3.881579,3.881579,3.881579,3.881579,3.881579,3.881579,3.881579,3.881579,3.881579
3,3.0,3.566667,3.566667,3.566667,3.566667,3.566667,3.566667,5.0,3.566667,3.566667,...,3.566667,3.566667,3.566667,3.566667,3.566667,3.566667,3.566667,3.566667,3.566667,3.566667
4,4.052326,4.052326,4.052326,5.0,5.0,4.052326,4.052326,4.0,4.052326,4.052326,...,4.052326,4.052326,4.052326,4.052326,4.052326,4.052326,4.052326,4.052326,4.052326,4.052326
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
287,3.651515,4.0,3.651515,4.0,4.0,3.651515,3.0,3.0,5.0,3.0,...,3.651515,3.651515,3.651515,3.651515,3.651515,3.651515,3.651515,3.651515,3.651515,3.651515
288,3.0,3.365385,3.0,3.0,3.365385,3.365385,3.365385,3.365385,4.0,3.365385,...,3.365385,3.0,3.365385,3.0,3.365385,3.365385,3.365385,3.0,3.365385,3.365385
289,3.955307,3.955307,3.955307,4.5,4.5,3.955307,3.955307,4.0,3.955307,3.955307,...,3.955307,3.955307,3.955307,3.955307,3.955307,3.955307,4.0,3.955307,2.5,3.955307


In [186]:
ui_df = uu_pivot.dot(ui_pivot)
ui_df

item,0,1,2,3,4,5,6,7,8,9,...,443,444,445,446,447,448,449,450,451,452
user_1,,,,,,,,,,,,,,,,,,,,,
0,3.493151,4.214164,3.714286,4.125397,4.253382,3.718426,3.571343,4.445946,3.914286,3.293151,...,3.718426,3.67248,3.904262,3.550208,3.718426,3.718426,3.718426,2.704262,3.718426,3.718426
1,3.527165,4.01838,3.719068,4.086316,4.3,3.721316,4.01838,3.997753,3.835,3.132753,...,3.890888,3.847792,3.761476,3.81838,2.728723,4.061476,3.612792,3.41838,3.847792,3.847792
2,3.547629,3.547629,3.579705,4.063105,4.467925,3.619058,3.391677,3.995181,3.663105,2.991677,...,3.536792,3.680839,3.50822,3.30822,3.50822,3.50822,3.50822,3.536792,3.241611,3.536792
3,3.335357,3.540189,3.312607,4.496826,4.404607,3.523107,3.355989,3.404707,3.615207,2.933781,...,3.681665,3.109208,3.740708,3.681665,3.681665,3.681665,3.681665,3.681665,3.681665,3.530208
4,3.797403,4.178795,4.08886,4.9,4.662338,4.01386,4.01386,4.290909,3.962338,2.897403,...,4.01386,3.778795,3.597403,3.778795,4.01386,3.88886,4.18886,3.778795,3.88886,4.01386
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
287,3.441299,4.299879,3.948148,4.148148,4.087245,3.541177,3.245946,4.245946,4.148148,3.441299,...,3.64832,3.602374,3.600293,3.394388,3.64832,3.64832,3.64832,3.148442,3.64832,3.500172
288,3.7947,4.108224,3.408444,3.9992,3.72926,3.474684,3.540124,4.13826,4.3855,2.5677,...,3.572077,3.592072,3.842017,2.826853,3.572077,3.572077,3.572077,3.027263,3.572077,3.572077
289,3.357758,3.494945,3.479708,4.6093,4.493101,3.397395,3.342814,3.522676,3.412926,2.297526,...,3.210373,3.624892,3.347186,3.484373,3.484373,3.347186,3.676436,3.484373,3.347186,3.484373
